In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pympc.models.boxatlas_parametric import BoxAtlas, MovingLimb, FixedLimb, Trajectory
from pympc.control import MPCHybridController
from pympc.feasible_set_library import FeasibleSetLibrary, load_library
from pympc.geometry.polytope import Polytope
import pympc.plot as mpc_plt

# Construction of Box-Atlas

Robot moving limbs

In [ ]:
from pympc.models.boxatlas_parameters import weight, stiffness
penetration = weight / stiffness

# right hand
A_rh = {
    'not_in_contact': np.array([[-1., 0.],[0., -1.]]),
    'in_contact': np.array([[1., 0.],[-1., 0.],[0., -1.]]),
}
b_rh = {
    'not_in_contact': np.array([[.4],[.5]]),
    'in_contact': np.array([[-.4],[.4+penetration],[.5]]),
}
contact_surfaces_rh = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_rh = np.array([[-.3], [.0]])
parametric_wall_distance = {
    'label': 'wall_distance_rh',
    'not_in_contact': {'index': [0], 'coefficient': [1.]},
    'in_contact': {'index': [0, 1], 'coefficient': [-1., 1.]},
    'min': -.05,
    'max': .05,
} # if positive moves the wall away from the body
parameters_rh = [parametric_wall_distance]
rh = MovingLimb(A_rh, b_rh, contact_surfaces_rh, q_rh, parameters=parameters_rh)

# left foot
A_lf = {
    'not_in_contact': np.array([[0., -1.]]),
    'in_contact': np.array([[0., 1.],[0., -1.]]),
}
b_lf = {
    'not_in_contact': np.array([[.5]]),
    'in_contact': np.array([[-.5],[.5+2.*penetration]]),
}
contact_surfaces_lf = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_lf = np.array([[.2], [-.5 - penetration / 2.]])
lf = MovingLimb(A_lf, b_lf, contact_surfaces_lf, q_lf)

# right foot
A_rf = {
    'not_in_contact': np.array([[-1., 0.],[0., -1.]]),
    'in_contact': np.array([[0., 1.],[-1., 0.],[0., -1.]]),
}
b_rf = {
    'not_in_contact': np.array([[.5],[.5]]),
    'in_contact': np.array([[-.5],[.5],[.5+2.*penetration]]),
}
contact_surfaces_rf = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_rf = np.array([[-.2], [-.5 - penetration / 2.]])
rf = MovingLimb(A_rf, b_rf, contact_surfaces_rf, q_rf)

Assemble robot

In [ ]:
limbs = { 'moving': {'rh': rh, 'lf': lf, 'rf': rf}, 'fixed': {} }
nominal_mode = {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}
box_atlas = BoxAtlas(limbs, nominal_mode)

Print state, input, and modes

In [ ]:
box_atlas.print_state_labels()
box_atlas.print_input_labels()
print 'Box-atlas modes:\n', box_atlas.contact_modes

# Hybrid MPC Controller

Hybrid MPC controller

In [ ]:
N = 10
objective_norm = 'two'
controller = MPCHybridController(
    box_atlas.pwa_system,
    N,
    objective_norm,
    box_atlas.Q,
    box_atlas.R,
    box_atlas.P,
    box_atlas.X_N)

Trick to avoid forbidden transitions

In [ ]:
controller = box_atlas.avoid_forbidden_transitions(controller)

## Visualizer

In [ ]:
# ['wall_distance_rh', 'qlfx', 'qlfy', 'qrfx', 'qrfy', 'qrhx', 'qrhy', 'qbx', 'qby', 'tb', 'vbx', 'vby', 'ob']
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = .05
x_0[10,0] = -1.5
u, x, ss = controller.feedforward(x_0)[0:3]

In [ ]:
from pympc.models.boxatlas_parameters import sampling_time

box_atlas.visualize(x[0])
time.sleep(10 * sampling_time)
for k in range(N):
    box_atlas.visualize(x[k])
    time.sleep(sampling_time)

In [ ]:
print ss
box_atlas.print_mode_sequence(ss)